# IMPORT

In [ ]:
# Model
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch import optim
from torch.optim.lr_scheduler import ExponentialLR 

# Others
import pandas as pd
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
import os
import sys
import json

from training.train import train
from preprocessing.preprocessing import *
from dataset.thermal_dataset import thermal_dataset
from utils.csv_utils import *
from models.complete_model import complete_model
from models.complete_model_autoencoder import complete_model_autoencoder
from models.lstm_only import lstm_only
from testing.test import test
prefix = '.'
    
pd.set_option('display.max_columns', None)

**Reproducibility and Device Setting**

In [ ]:
# Set a seed for reproducibility purposes
seed = 46
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)

# Set the device (cuda for Nvidia GPUs, mps for M1, M2 .. Apple Silicon)
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

# Test

In [ ]:
# Need to specify a model.pt file to load! In the directory results that you want to test
result_path = '/results/training_2025-03-05_13-29'  
model_path = prefix + result_path +'/model.pt'

# Loss function
loss_fn = nn.MSELoss()

# Read hyperparameters
with open (prefix + result_path + '/hyperparam.json', 'r') as f:
    hyperparams = json.load(f)
   
if hyperparams['norm']:
    X = np.load(prefix+'/data/X'+'testing'+str(hyperparams['file'])+'.npy')
    Y = np.load(prefix+'/data/Y'+str(hyperparams['window_size'])+'testing'+str(hyperparams['file'])+'.npy')
else:
    X = np.load(prefix+'/data/X'+'testing'+str(hyperparams['file'])+'.npy')
    Y = np.load(prefix+'/data/Y'+str(hyperparams['window_size'])+'testing'+str(hyperparams['file'])+'.npy')
print('X',X)
print('Y',Y)
datasetTest = thermal_dataset((X,Y), hyperparams['timesteps'], device)
loaderTest = DataLoader(datasetTest, shuffle=True, batch_size=hyperparams['batch_size'])
# Section in which we reinitialize the model for testing depending on the hyperparameters.json and ensemble.json (if present) in the directories of results
# Read Ensemble models data (if complete = True)
if hyperparams['ensemble'] == True:
    # Read Ensemble model parameters and initialize the model
    with open (prefix + result_path + '/ensemble.json', 'r') as f:
        model_dict = json.load(f)
    
    if hyperparams['extractor_type']=='conv':
        print("HERE")
        with open(prefix + result_path + '/autoencoder.json', 'r') as f:
            autoencoder_dict = json.load(f)
        model = complete_model_autoencoder(
            model_dict=model_dict,
            device=device,
            autoencoder_dim=autoencoder_dict['in_kern_out'],
            pooling_kernel_size=autoencoder_dict['pooling_kernel_size'],
            padding=autoencoder_dict['padding'],
            pooling=autoencoder_dict['pooling'],
            timesteps=hyperparams['timesteps'],
            scale_factor=autoencoder_dict['scale_factor'],
            upsample_mode=autoencoder_dict['upsample_mode'],
            mode=hyperparams['mode'],
            heterogeneous=hyperparams['heterogeneous'],
            norm=hyperparams['norm']
        ).to(device)
    elif hyperparams['extractor_type']=='lstm':
        with open(prefix + result_path + '/autoencoder.json', 'r') as f:
            autoencoder_dict = json.load(f)
        model = complete_model_autoencoder(
                                    model_dict=model_dict, 
                                    device=device, 
                                    timesteps=hyperparams['timesteps'],
                                    autoencoder_dim=autoencoder_dict['in_hidd'], 
                                    dropout=autoencoder_dict['dropout'],
                                    extractor_type=hyperparams['extractor_type'],
                                    heterogeneous=hyperparams['heterogeneous'],
                                    norm=hyperparams['norm']
                                    ).to(device)
    else:
        model = complete_model(
            hidden_dim=hyperparams['hidden_dim'], 
            input_dim=4, 
            model_dict=model_dict, 
            device=device, 
            num_layers=hyperparams['num_layers'], 
            mode=hyperparams['mode'], 
            extractor_type=hyperparams['extractor_type']).to(device)

# Otherwise create an LSTM only model
else:
    model = lstm_only(
        hidden_dim=hyperparams['hidden_dim'], 
        timesteps=hyperparams['timesteps'] ,
        input_dim=4, 
        output_dim=3).to(device)

print(hyperparams)
#print("state_dict",model.state_dict().keys())


test_model_metrics, test_autoencoder_metrics, loss_model_avg, loss_autoencoder_avg, total_inference_time, inference_time_avg, y_true_list, y_pred_list, x_true_list, x_pred_list = test(
    model=model,
    model_path=model_path,
    test_dataloader=loaderTest,
    loss_fn=loss_fn,
    complete=hyperparams['ensemble'],
    autoencoder= hyperparams['extractor_type'] == 'conv' or hyperparams['extractor_type'] == 'lstm',
    autoencoder_only=True,
    device=device
)

#train_val = True
print('Legend metrics:[X1,Y1,Z1]')
print("Test Model metrics:")
print(f"rmse: {test_model_metrics['rmse'][0]} , mae: {test_model_metrics['mae'][0]}, r2: {test_model_metrics['r2'][0]}")
print("Reference metrics(wrt. 0):")
print(f"rmse: {test_model_metrics['rmse_ref'][0]} , mae: {test_model_metrics['mae_ref'][0]}, r2: {test_model_metrics['r2_ref'][0]}")
print("Test Autoencoder metrics:")
print(f"rmse: {test_autoencoder_metrics['rmse'][0]} , mae: {test_autoencoder_metrics['mae'][0]}, r2: {test_autoencoder_metrics['r2'][0]}")
print("Others:")
print(f"Model Average Loss: {loss_model_avg}, Autoencoder Average Loss: {loss_autoencoder_avg}, Total Inference Time (All Dataset): {total_inference_time}, Average Inference Time: {inference_time_avg}")
if hyperparams['ensemble'] == True:
    if not 'NoOpModule' in str(type(model.ensemble)):
        print(f"Weights of the Ensemble models: {model.ensemble.weights.cpu().tolist()}")

In [ ]:
path = prefix + result_path  # type ./results/ensemble_batch_256

# Load the dictionaries
with open (path + '/train_model_metrics.json', 'r') as f:
    train_metrics = json.load(f)
with open (path + '/valid_model_metrics.json', 'r') as f:
    valid_metrics = json.load(f)

# Initialize a list of epochs [1,2,3, ...] for the plots x-axis
epochs = [i for i in range(1, hyperparams['num_epochs']+1)]

# Compute training loss as the square of rmse for each element because loss = mse
# Both for training and validation
training_loss = train_metrics['loss']
valid_loss = valid_metrics['loss'] 

fig1, ax1 = plt.subplots()

ax1.plot(epochs, training_loss, label='Training', color='b')
ax1.plot(epochs, valid_loss, label='Validation', color='r')
ax1.set_title('Loss (RMSE^2)')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Loss value')
ax1.legend()

plt.savefig(path+'/loss')  # Save the figure to the specified path
plt.show()

In [ ]:
#print(len(x_pred_list[0][0]))

#list is made like [number of sample windows][number of sensors][number of timesteps] [7472][3][199]
x_pred_sensor_1 = x_pred_list[0][0].copy()
x_pred_sensor_2 = x_pred_list[0][1].copy()
x_pred_sensor_3 = x_pred_list[0][2].copy()
x_pred_sensor_4 = x_pred_list[0][3].copy()
# we unroll the sliding windows to get the predictions as a continuous list because the windows are overlapped
x_pred_sensor_1 += [x_pred[0][-1] for x_pred in x_pred_list[1:]]
x_pred_sensor_2 += [x_pred[1][-1] for x_pred in x_pred_list[1:]]
x_pred_sensor_3 += [x_pred[2][-1] for x_pred in x_pred_list[1:]]
x_pred_sensor_4 += [x_pred[3][-1] for x_pred in x_pred_list[1:]]

x_true_sensor_1 = x_true_list[0][0].copy() 
x_true_sensor_2 = x_true_list[0][1].copy()
x_true_sensor_3 = x_true_list[0][2].copy()
x_true_sensor_4 = x_true_list[0][3].copy()
# we unroll the sliding windows to get the predictions as a continuous list because the windows are overlapped
x_true_sensor_1 += [x_true[0][-1] for x_true in x_true_list[1:]]
x_true_sensor_2 += [x_true[1][-1] for x_true in x_true_list[1:]]
x_true_sensor_3 += [x_true[2][-1] for x_true in x_true_list[1:]]
x_true_sensor_4 += [x_true[3][-1] for x_true in x_true_list[1:]]

print(len(x_true_sensor_3))
print(len(x_pred_sensor_3))

In [ ]:
path = prefix + result_path  # type ./results/ensemble_batch_256

# Load the dictionaries
with open (path + '/train_model_metrics.json', 'r') as f:
    train_metrics = json.load(f)
with open (path + '/valid_model_metrics.json', 'r') as f:
    valid_metrics = json.load(f)

# Initialize a list of epochs [1,2,3, ...] for the plots x-axis
epochs = [i for i in range(1, hyperparams['num_epochs']+1)]

fig2, ax2 = plt.subplots()

ax2.plot(epochs, [mae[1] for mae in train_metrics['mae']], label='Training', color='b')
ax2.plot(epochs, [mae[1] for mae in valid_metrics['mae']], label='Validation', color='r')
ax2.set_title('Mean Absolute Error')
ax2.set_xlabel('Epochs')
ax2.set_ylabel('MAE value Y')
ax2.legend()

plt.savefig(path + '/mae')  # Save the figure to the specified path
plt.show()

In [ ]:
path = prefix + result_path  # type ./results/ensemble_batch_256

# Load the dictionaries
with open (path + '/train_model_metrics.json', 'r') as f:
    train_metrics = json.load(f)
with open (path + '/valid_model_metrics.json', 'r') as f:
    valid_metrics = json.load(f)

# Initialize a list of epochs [1,2,3, ...] for the plots x-axis
epochs = [i for i in range(1, hyperparams['num_epochs']+1)]

fig3, ax3 = plt.subplots()

ax3.plot(epochs, [r2[1] for r2 in train_metrics['r2']], label='Training', color='b')
ax3.plot(epochs, [r2[1] for r2 in valid_metrics['r2']], label='Validation', color='r')
ax3.set_title('R2 (Coefficient of determination)')
ax3.set_xlabel('Epochs')
ax3.set_ylabel('R2 value Y')
plt.legend()

plt.savefig(path + '/r2')  # Save the figure to the specified path
plt.show()

In [ ]:
path = prefix + result_path

# Create the lists of X1, Y1, Z1 true values to plot
x1_true = [x[0] for x in y_true_list]
y1_true = [y[1] for y in y_true_list]
z1_true = [z[2] for z in y_true_list]

# Create the lists of X1, Y1, Z1 predicted values to plot
x1_pred = [x[0] for x in y_pred_list]
y1_pred = [y[1] for y in y_pred_list]
z1_pred = [z[2] for z in y_pred_list]

# Create x-axis
t = [i for i in range(1, len(y_pred_list)+1)]


In [ ]:
fig4, ax4 = plt.subplots()

ax4.plot(t, y1_true, label='Ground Truth', color='b')
ax4.plot(t, x1_pred, label='Prediction', color='r')
ax4.set_title('X1 Sequence')
ax4.set_xlabel('Time')
ax4.set_ylabel('X1')
plt.legend()

plt.savefig(path + '/X1')  # Save the figure to the specified path
plt.show()

In [ ]:
# Initialize a list of epochs [1,2,3, ...] for the plots x-axis

fig5, ax5 = plt.subplots()

ax5.plot(t, y1_true, label='Ground Truth', color='b')
ax5.plot(t, y1_pred, label='Prediction', color='r')
ax5.set_title('Y1 Sequence')
ax5.set_xlabel('Time')
ax5.set_ylabel('Y1')
plt.legend()

plt.savefig(path + '/Y1')  # Save the figure to the specified path
plt.show()


In [ ]:
# Initialize a list of epochs [1,2,3, ...] for the plots x-axis
fig6, ax6 = plt.subplots()

ax6.plot(t, z1_true, label='Ground Truth', color='b')
ax6.plot(t, z1_pred, label='Prediction', color='r')
ax6.set_title('Z1 Sequence ')
ax6.set_xlabel('Time [1 u = 11 s]')
ax6.set_ylabel('Z1')
plt.legend()

plt.savefig(path + '/Z1')  # Save the figure to the specified path
plt.show()

In [ ]:
#print(len(x_pred_list[0][0]))

#list is made like [number of sample windows][number of sensors][number of timesteps] [7472][3][199]
x_pred_sensor_1 = x_pred_list[0][0].copy()
x_pred_sensor_2 = x_pred_list[0][1].copy()
x_pred_sensor_3 = x_pred_list[0][2].copy()
x_pred_sensor_4 = x_pred_list[0][3].copy()
# we unroll the sliding windows to get the predictions as a continuous list because the windows are overlapped
x_pred_sensor_1 += [x_pred[0][-1] for x_pred in x_pred_list[1:]]
x_pred_sensor_2 += [x_pred[1][-1] for x_pred in x_pred_list[1:]]
x_pred_sensor_3 += [x_pred[2][-1] for x_pred in x_pred_list[1:]]
x_pred_sensor_4 += [x_pred[3][-1] for x_pred in x_pred_list[1:]]

x_true_sensor_1 = x_true_list[0][0].copy() 
x_true_sensor_2 = x_true_list[0][1].copy()
x_true_sensor_3 = x_true_list[0][2].copy()
x_true_sensor_4 = x_true_list[0][3].copy()
# we unroll the sliding windows to get the predictions as a continuous list because the windows are overlapped
x_true_sensor_1 += [x_true[0][-1] for x_true in x_true_list[1:]]
x_true_sensor_2 += [x_true[1][-1] for x_true in x_true_list[1:]]
x_true_sensor_3 += [x_true[2][-1] for x_true in x_true_list[1:]]
x_true_sensor_4 += [x_true[3][-1] for x_true in x_true_list[1:]]

# Create x-axis
t_x = [i for i in range(1, len(x_true_sensor_4)+1)]

In [ ]:
fig7, ax7 = plt.subplots()

ax7.plot(t_x, x_true_sensor_1, label='Ground Truth', color='b')
ax7.plot(t_x, x_pred_sensor_1, label='Prediction', color='r')
ax7.set_title('Autoencoder Sensor 1 Prediction')
ax7.set_xlabel('Time [1 u = 11 s]')
ax7.set_ylabel('Temp1 [°C]')
plt.legend()

plt.savefig(path + '/temp1')  # Save the figure to the specified path
plt.show()

In [ ]:
fig8, ax8 = plt.subplots()

ax8.plot(t_x, x_true_sensor_2, label='Ground Truth', color='b')
ax8.plot(t_x, x_pred_sensor_2, label='Prediction', color='r')
ax8.set_title('Autoencoder Sensor 2 Prediction')
ax8.set_xlabel('Time [1 u = 11 s]')
ax8.set_ylabel('Temp2 [°C]')
plt.legend()

plt.savefig(path + '/temp2')  # Save the figure to the specified path
plt.show()

In [ ]:
fig9, ax9 = plt.subplots()

ax9.plot(t_x, x_true_sensor_3, label='Ground Truth', color='b')
ax9.plot(t_x, x_pred_sensor_3, label='Prediction', color='r')
ax9.set_title('Autoencoder Sensor 3 Prediction')
ax9.set_xlabel('Time [1 u = 11 s]')
ax9.set_ylabel('Temp3 [°C]')
plt.legend()

plt.savefig(path + '/temp3')  # Save the figure to the specified path
plt.show()

In [ ]:
fig10, ax10 = plt.subplots()

ax10.plot(t_x, x_true_sensor_4, label='Ground Truth', color='b')
ax10.plot(t_x, x_pred_sensor_4, label='Prediction', color='r')
ax10.set_title('Autoencoder Sensor 4 Prediction')
ax10.set_xlabel('Time [1 u = 11 s]')
ax10.set_ylabel('Temp4 [°C]')
plt.legend()

plt.savefig(path + '/temp4')  # Save the figure to the specified path
plt.show()

In [ ]:
fig10, ax10 = plt.subplots()

ax10.plot(t_x, x_true_sensor_4, label='Ground Truth', color='b')
ax10.plot(t_x, x_pred_sensor_4, label='Prediction', color='r')
ax10.set_title('Autoencoder Sensor 4 Prediction')
ax10.set_xlabel('Time')
ax10.set_ylabel('Temp4')
plt.legend()

plt.savefig(path + '/temp4')  # Save the figure to the specified path
plt.show()